In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report

from sktime.classification.kernel_based import RocketClassifier

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

2023-05-11 14:47:34.442771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/nzuchna/opt/anaconda3/envs/gameplay_kaggel_env/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


ImportError: cannot import name 'type_spec_registry' from 'tensorflow.python.framework' (/Users/nzuchna/opt/anaconda3/envs/gameplay_kaggel_env/lib/python3.9/site-packages/tensorflow/python/framework/__init__.py)

In [2]:
# get working directory and remove last folder
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print("Working Directory: ", os.getcwd())

Working Directory:  /Users/nzuchna/Desktop/Drive/2. Areas/University/Master-TUC/M2/4_Forschungsmodul/student-performance


## Attempts to make a joint of data and labels

df_0_4 = pd.read_csv('data/processed/df_0_4_flattened.csv')
labels = pd.read_csv('data/processed/labels.csv')

df_0_4.head(10)

df_edit = df_0_4.drop(columns=['Unnamed: 0', 'level_group_1'])
df_edit.head()

#df_edit = df_edit[~df_edit['session_id_1'].isin(labels.session)]
#labels = labels[~labels['session'].isin(df_edit.session_id_1)]

labels_q1_3 = labels[labels['q'] <= 3]

print(df_edit.shape, labels_q1_3.shape)

X1 = df_edit.drop(columns=['session_id_1'])
y1 = labels.drop(columns=['session', 'q'])

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, train_size=.75) # , test_size=0.5
print(X_train1.shape, y_train1.shape, X_test1.shape, y_test1.shape)

In [7]:
# X
df = pd.read_csv("data/processed/df_5_12_flattened.csv", index_col=0)
X = df.iloc[:, 2:].values
# X = np.delete(X, 61, 1)

# y
labels = pd.read_csv("data/processed/labels_q4-13.csv")
y = labels['correct'][labels['q'] == 10].values

# Creating training and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify = y)

# Feature Scaling
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

def eval_model(model, cm_title: str, ):
    # Model evalutation
    y_pred = np.round(model.predict(X_test))
    # print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

    # fmt='d' formats the numbers as digits, which means integers
    cm = confusion_matrix(y_test, y_pred)
    hp = sns.heatmap(cm, annot=True, fmt='d')
    hp.set_title(f"Confusion matrix of {cm_title}")
    hp.set(xlabel='Predicted', ylabel='Actual')

    print(classification_report(y_test,y_pred))

In [5]:
%%time
rocket1 = RocketClassifier()

rocket1.fit(X_train, y_train)

rocket_preds = rocket1.predict(X_test)
print("Rocket1 Accuracy: " + str(metrics.accuracy_score(y_test, rocket_preds)))

Rocket1 Accuracy: 0.5054463148960249
CPU times: user 55min 49s, sys: 1min 23s, total: 57min 12s
Wall time: 29min 14s


In [ ]:
eval_model(rocket1, 'Rocket1')

In [6]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(16493, 170) (16493,) (7069, 170) (7069,)


In [11]:
# Create a simple neural network model
model = Sequential()
model.add(Dense(64, input_dim=170, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
test_loss, test_mse = model.evaluate(X_test, y_test)

print(f'Test loss: {test_loss}, Test MSE: {test_mse}')

eval_model(model, 'Neural Net 1')

NameError: name 'Sequential' is not defined

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=170))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)

eval_model(model, 'Neural Net 4 - with Dropout (chatGPT)')